In [1]:
import numpy as np 
import pandas as pd

from gsdmm import MovieGroupProcess

from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim
from gensim.models.ldamulticore import LdaMulticore
import re
import spacy

from gensim.models import Phrases
from gensim.models.phrases import Phraser

C:\Users\Alkan\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
df=pd.read_csv('sdg_preprocessed.csv')

In [9]:
df

,Unnamed: 0,date,Tweet,labels
0,0,2021-10-15 00:05:27+00:00,çinde den e kadar asgari ücret kodlaması var ...,negative
1,1,2021-10-15 00:11:08+00:00,aman aman önemli işler yapıyorsunuzeytasgari ü...,negative
2,2,2021-10-15 00:22:51+00:00,asgari ücret yıla bedel olacak,negative
3,3,2021-10-15 00:23:21+00:00,asgari ücret,negative
4,4,2021-10-15 00:31:42+00:00,dolar olmuş ülkede tek zam gelmeyen asgari üc...,negative
...,...,...,...,...
749182,749182,2023-02-12 22:57:51+00:00,NaN,negative
749183,749183,2023-02-12 23:04:45+00:00,ekonomiyi çökerttiler derken ak parti öncesind...,negative
749184,749184,2023-02-12 23:13:45+00:00,yandaş medya el ele vermiş emekliye gün doğdu ...,negative
749185,749185,2023-02-12 23:21:58+00:00,özel okul öğretmenlerinin asgari ücret altına ...,negative


In [12]:
negative = df[df["labels"] == "negative"]

In [13]:
negative

,Unnamed: 0,date,Tweet,labels
0,0,2021-10-15 00:05:27+00:00,çinde den e kadar asgari ücret kodlaması var ...,negative
1,1,2021-10-15 00:11:08+00:00,aman aman önemli işler yapıyorsunuzeytasgari ü...,negative
2,2,2021-10-15 00:22:51+00:00,asgari ücret yıla bedel olacak,negative
3,3,2021-10-15 00:23:21+00:00,asgari ücret,negative
4,4,2021-10-15 00:31:42+00:00,dolar olmuş ülkede tek zam gelmeyen asgari üc...,negative
...,...,...,...,...
749182,749182,2023-02-12 22:57:51+00:00,NaN,negative
749183,749183,2023-02-12 23:04:45+00:00,ekonomiyi çökerttiler derken ak parti öncesind...,negative
749184,749184,2023-02-12 23:13:45+00:00,yandaş medya el ele vermiş emekliye gün doğdu ...,negative
749185,749185,2023-02-12 23:21:58+00:00,özel okul öğretmenlerinin asgari ücret altına ...,negative


In [14]:
def make_n_grams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)  # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    bigrams_text = [bigram_mod[doc] for doc in texts]
    trigrams_text =  [trigram_mod[bigram_mod[doc]] for doc in bigrams_text]
    return trigrams_text

In [15]:
negative['Tweet'] = negative.Tweet.values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))

C:\Users\Alkan\AppData\Local\Temp\ipykernel_23968\3630039086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative['Tweet'] = negative.Tweet.values.tolist()


In [16]:
tokens_tweets = list(sent_to_words(negative['Tweet']))

In [17]:
np.random.seed(0)

In [18]:
mgp = MovieGroupProcess(K=10, alpha=0.01, beta=0.01, n_iters=30) 

vocab = set(x for tweet in tokens_tweets for x in tweet)
n_terms = len(vocab)
model = mgp.fit(tokens_tweets, n_terms)

In stage 0: transferred 451642 clusters with 10 clusters populated
In stage 1: transferred 328440 clusters with 10 clusters populated
In stage 2: transferred 232428 clusters with 10 clusters populated
In stage 3: transferred 166609 clusters with 10 clusters populated
In stage 4: transferred 134813 clusters with 10 clusters populated
In stage 5: transferred 119426 clusters with 10 clusters populated
In stage 6: transferred 111350 clusters with 10 clusters populated
In stage 7: transferred 106239 clusters with 10 clusters populated
In stage 8: transferred 101768 clusters with 10 clusters populated
In stage 9: transferred 98394 clusters with 10 clusters populated
In stage 10: transferred 96164 clusters with 10 clusters populated
In stage 11: transferred 94495 clusters with 10 clusters populated
In stage 12: transferred 93335 clusters with 10 clusters populated
In stage 13: transferred 92360 clusters with 10 clusters populated
In stage 14: transferred 90917 clusters with 10 clusters popula

In [19]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster,sort_dicts))

In [20]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-15:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 20)

Number of documents per topic : [58973 42404 15873 91373 67455 50254 57611 31960 62494 78619]

Most important clusters (by number of docs inside): [3 9 4 8 0 6 5 1 7 2]

Cluster 3 : [('asgari', 84543), ('ücret', 83400), ('bir', 26245), ('tl', 14513), ('kadar', 11164), ('yok', 8241), ('emekli', 8004), ('asgariücret', 7936), ('bin', 7909), ('var', 7679), ('olsun', 7555), ('mi', 7164), ('zam', 7152), ('maaş', 7097), ('değil', 6925), ('bile', 6448), ('maaşı', 6356), ('açlık', 5804), ('altında', 5307), ('lira', 4338)]

Cluster 9 : [('asgari', 79898), ('ücret', 77706), ('bir', 28054), ('tl', 24382), ('bin', 11972), ('var', 9944), ('ev', 9414), ('kadar', 8847), ('yok', 8837), ('ben', 8225), ('bile', 7960), ('lira', 7279), ('kira', 6216), ('mi', 6204), ('maaş', 5831), ('değil', 5687), ('para', 5487), ('sen', 5211), ('iş', 4202), ('aylık', 4136)]

Cluster 4 : [('asgari', 63924), ('ücret', 60936), ('zam', 27312), ('tl', 19193), ('bir', 13213), ('bin', 8388), ('sonra', 7591), ('dolar', 7334), ('a